In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        # print(os.path.join(dirname, filename))
        pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [40]:
import os

import torch
import torch.nn as nn
import torch.optim as optim
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.distributed import DistributedSampler

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from tqdm import tqdm
import gc
import time

print("imports done!")

imports done!


In [3]:
path = "../input/digit-recognizer/"
print(os.listdir(path))

['sample_submission.csv', 'train.csv', 'test.csv']


In [5]:
train_path = path+"train.csv"
test_path = path+"test.csv"

train_df = pd.read_csv(train_path)
train_df

# test_df = pd.read_csv(test_path)
# test_df

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
X = train_df.drop(["label"], axis=1)
y = train_df[["label"]]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, 
                                                  stratify=y)

print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

(33600, 784)
(8400, 784)
(33600, 1)
(8400, 1)


In [26]:
torch.tensor(X_train.reset_index(drop=True).values)[0].shape

torch.Size([784])

In [36]:
def fn():
    a = torch.tensor(X_train.reset_index(drop=True).values).reshape(-1, 1, 28, 28)
    b = torch.tensor(y_train.reset_index(drop=True).values).long().reshape(-1, 1)
    print(a[0].shape, b[0].shape, len(a), sep='\n')


fn()

torch.Size([1, 28, 28])
torch.Size([1])
33600


In [37]:
class MNISTDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X.reset_index(drop=True).values).reshape(-1, 1, 28, 28)
        self.y = torch.tensor(y.reset_index(drop=True).values).long().reshape(-1, 1)
        
    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return self.X[index], self.y[index]


In [38]:
train_dataset = MNISTDataset(X_train, y_train)
val_dataset = MNISTDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

print("Dataset and DataLoader created!")

Dataset and DataLoader created!


In [16]:
print("no. of gpus:", torch.cuda.device_count())

no. of gpus: 2


In [17]:
torch.cuda.is_available()

True

In [18]:
if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        gpu_name = torch.cuda.get_device_name(i)
        print(f"GPU {i}: {gpu_name}")

GPU 0: Tesla T4
GPU 1: Tesla T4


In [19]:
if torch.cuda.is_available():
    current_device_index = torch.cuda.current_device()
    current_device_name = torch.cuda.get_device_name(current_device_index)
    print(f"Currently selected GPU: Device {current_device_index} - {current_device_name}")

Currently selected GPU: Device 0 - Tesla T4


In [41]:
# --- 1. Define a Simple Model ---
# A basic neural network
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.layer1 = nn.Linear(10, 50)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(50, 5)

    def forward(self, x):
        return self.layer2(self.relu(self.layer1(x)))


# --- 3. DDP Setup Function ---
# This function initializes the distributed process group
def setup(rank, world_size):
    """
    Initializes the distributed environment.
    'rank' is the unique ID for each process (GPU).
    'world_size' is the total number of processes (GPUs).
    """
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '12355' # A free port
    
    # Initialize the process group
    # 'nccl' is the fastest backend for NVIDIA GPUs
    dist.init_process_group(backend="nccl", rank=rank, world_size=world_size)
    
    # Set the device for this process
    torch.cuda.set_device(rank)


# --- 4. DDP Cleanup Function ---
def cleanup():
    """Destroys the process group."""
    dist.destroy_process_group()


# --- 5. The Main Training/Validation Function ---
# This is the function that will be run *by each process*
def train_and_validate(rank, world_size):
    print(f"Running DDP on rank {rank} (GPU {rank}).")
    
    # --- Setup ---
    setup(rank, world_size)

    # --- Model ---
    # Create the model and move it to the correct GPU (device=rank)
    model = SimpleNN().to(rank)
    # Wrap the model with DDP
    # This handles the gradient synchronization
    ddp_model = DDP(model, device_ids=[rank])

    # --- Data ---
    train_dataset = RandomDataset(size=10, length=1024)
    val_dataset = RandomDataset(size=10, length=128)

    # The DistributedSampler is crucial!
    # It splits the dataset across the GPUs so they don't use the same data
    train_sampler = DistributedSampler(train_dataset, num_replicas=world_size, rank=rank)
    val_sampler = DistributedSampler(val_dataset, num_replicas=world_size, rank=rank, shuffle=False)

    # The DataLoader uses the sampler
    # Note: shuffle=False because the sampler handles shuffling
    train_loader = DataLoader(train_dataset, batch_size=32, sampler=train_sampler, shuffle=False)
    val_loader = DataLoader(val_dataset, batch_size=32, sampler=val_sampler, shuffle=False)

    # --- Loss and Optimizer ---
    criterion = nn.CrossEntropyLoss().to(rank)
    optimizer = optim.SGD(ddp_model.parameters(), lr=0.01)

    num_epochs = 5

    for epoch in range(num_epochs):
        # **IMPORTANT**: Set the epoch for the sampler
        # This ensures proper shuffling across epochs
        train_sampler.set_epoch(epoch)
        
        # --- Training ---
        ddp_model.train()
        total_train_loss = 0.0
        
        for inputs, labels in train_loader:
            # Move data to the process's assigned GPU
            inputs, labels = inputs.to(rank), labels.to(rank)

            optimizer.zero_grad()
            outputs = ddp_model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            total_train_loss += loss.item()

        # --- Metric Aggregation ---
        # To get the *total* loss across all GPUs, we must synchronize
        # We create a tensor on the GPU and use all_reduce to sum them up
        total_train_loss_tensor = torch.tensor(total_train_loss).to(rank)
        dist.all_reduce(total_train_loss_tensor, op=dist.ReduceOp.SUM)
        
        # Only print from the main process (rank 0) to avoid clutter
        if rank == 0:
            avg_train_loss = total_train_loss_tensor.item() / world_size
            print(f"Epoch {epoch+1}/{num_epochs}, "
                  f"Avg. Training Loss: {avg_train_loss / len(train_loader):.4f}")

        # --- Validation ---
        ddp_model.eval()
        total_val_loss = 0.0
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(rank), labels.to(rank)
                outputs = ddp_model(inputs)
                loss = criterion(outputs, labels)
                total_val_loss += loss.item()

        # Aggregate validation loss
        total_val_loss_tensor = torch.tensor(total_val_loss).to(rank)
        dist.all_reduce(total_val_loss_tensor, op=dist.ReduceOp.SUM)

        if rank == 0:
            avg_val_loss = total_val_loss_tensor.item() / world_size
            print(f"Epoch {epoch+1}/{num_epochs}, "
                  f"Avg. Validation Loss: {avg_val_loss / len(val_loader):.4f}")

    # --- Cleanup ---
    cleanup()


# --- 6. The Spawner ---
# This function spawns the 'train_and_validate' function on all available GPUs
def main():
    world_size = torch.cuda.device_count()
    if world_size < 2:
        print("Requires at least 2 GPUs to run DDP.")
        return
        
    print(f"Using {world_size} GPUs.")
    
    # start the timer
    start_time = time.time()

    # mp.spawn will create 'world_size' processes and run the
    # 'train_and_validate' function in each one, passing in
    # 'rank' (from 0 to world_size-1) and 'world_size'.
    mp.spawn(train_and_validate,
             args=(world_size,),
             nprocs=world_size,
             join=True)

    # end the timer
    end_time = time.time()
    print(f"Training completed in {end_time - start_time:.2f} seconds.")

if __name__ == "__main__":
    main()

Using 2 GPUs.


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/usr/lib/python3.11/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/spawn.py", line 132, in _main
    self = reduction.pickle.load(from_parent)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'train_and_validate' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/usr/lib/python3.11/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/spawn.py", line 132, in _main
    self = reduction.pickle.load(from_parent)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'train_and_validate' on <module '__main__' (built-in)>
W1031 06:40:

ProcessExitedException: process 1 terminated with exit code 1